
L'obiettivo del progetto è apprendere mediante una rete neurale la trasformazione da punti del piano espressi in coordinate polari ad una rappresentazione basata su di una griglia discreta di dimensione 10x10, dove la cella della griglia ha valore 1 se contiene il punto, e 0 altrimenti.

Il dataset supervisionato è fornito in questo notebook nella forma di una generatore. Il generatore deve essere considerato come una "scatola nera" il cui comportamento deve essere appreso. 

Dovete progettare una rete neurale in grado di raggiungere una accuratezza del 95%. Questa è una condizione necessaria per superare l'esame, ma l'accuratezza non influisce in altro modo sulla valutazione.  

I modelli che raggiungono l'accuratezza attesa saranno invece valutati in modo inversamente proporzionale al numero dei loro parametri: **più il modello è piccolo, meglio è.**


**Attenzione**: Qualunque soluzione che tragga vantaggio, diretto o indiretto, da meta-conoscenza relativa al generatore sarà automaticamente bocciato.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model

2023-01-10 11:48:54.718306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Veniamo al generatore. Questo restituisce delle triple della forma
((theta,rho),out) dove (theta,rho) sono le coordinate polari di un punto nel primo quadrante del piano, e out è una mappa 10x10 con "1" in correspondenza alla cella che contiene il punto, e "0" altrimenti.

Settando  flat=True, la mappa 10x10 viene appiattita ad un vettore di dimensione 100. Potete utilizzare questa variante, se preferite. Nessuna altra modifica del generatore è ammessa. 

In [3]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Creiamo una istanza del generatore con una griglia di dimensione 3x4

In [ ]:
g1,g2 = 3,4
gen = polar_generator(1,grid=(g1,g2),noise=0.0)

... e osserviamo qualche esempio

In [ ]:
(theta,rho),maps = next(gen)
for i,map in enumerate(maps):
  #let us compute the cartesian coordinates
  x = np.cos(theta[i])*rho[i]
  y = np.sin(theta[i])*rho[i]
  print("x coordinate (row): {}".format(int(x*g1)))
  print("y coordinate (col): {}".format(int(y*g2)))
  print("map:")
  print(np.reshape(map,(g1,g2)))

x coordinate (row): 1
y coordinate (col): 1
map:
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]


Utile esercizio: aggiungete rumore al generatore e verificate l'effetto sulla
"ground truth".

# Cosa consegnare

Ai fini del progetto dovete lavorare con la **griglia di default di dimensione 10x10, e con il rumore di default .002**

il generatore deve essere trattato come una scatola nera: non modificatelo e non sfruttate la sua semantica, che si suppone ignota. Potete lavorare in modlaità "flat", se preferite.

Dovete:

1.   definire una funzione per il calcolo della accuratezza (potete prendere ispirazione dal cocice della cella precedente) 
2.   definire una rete neurale che prende in input theta e rho e restituisce out
3.  misurare l'accuratezza della rete, che deve essere maggiore o uguale del 95%; l'accuratezza deve essere misurata su almeno 20000 dati
4. perfezionare il modello cercando di diminuire il più possibile il numero dei parametri mantenendo una accuratezza superiore al 95%. Solo la vostra rete migliore deve essere consegnata.

Dovete consegnare un UNICO notebook eseguibile su colab, che contenga il codice della rete, il suo sommario con il numero dei parametri, la storia di training, il codice per il calcolo della accuratezza e la sua valutazione sulla vostra rete.

**N.B.** L'accuratezza deve essere superiore o uguale a 95%, ma non influisce in altro modo sulla valutazione. Il vostro punteggio dipenderà unicamente dal numero dei parametri: più è piccolo e più la vostra vaalutazione sarà elevata.  

#Buon lavoro!





In [114]:
import keras
import numpy as np
from keras import layers
from keras import activations
from keras import losses
from sklearn.model_selection import train_test_split
import keras.backend as K

size = 10

In [164]:
# def map_map(map: np.ndarray) -> np.ndarray:
#   out = np.zeros(size * 2)
#   point = np.argwhere(map==1)[0]
#   out[point[0]] = 1
#   out[size + point[1]] = 1
#   return out

def map_map(map: np.ndarray) -> np.ndarray:
  out = np.zeros(size * 2)
  point = np.argwhere(map==1)[0]
  return point

In [165]:
n_train = 100000
n_test = 20000

gen = polar_generator(n_train+n_test,grid=(size,size),noise=0.02)
(theta,rho),y = next(gen)
y = np.array(list(map(map_map, y)))
x=np.array([i for i in zip(theta,rho)])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=n_test/(n_train+n_test), shuffle=True, random_state=1)

In [166]:
def loss_fn(y_true, y_pred):
  point = np.array(y_true).argwhere(x==1)[1]

  x = K.mult(K.cos(y_true[:1]), y_true[:2])
  #x = np.cos(theta[i])*rho[i]
  #y = np.sin(theta[i])*rho[1,164

In [170]:
model = keras.Sequential([
    layers.Flatten(),
    layers.Dense(32, activation=activations.swish),
    layers.Dense(64, activation=activations.swish),
    layers.Dense(32, activation=activations.swish),

    layers.Dense(2, activation=activations.softmax)
])
model.build((None, 2))
model.summary()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_34 (Flatten)        (None, 2)                 0         
                                                                 
 dense_143 (Dense)           (None, 32)                96        
                                                                 
 dense_144 (Dense)           (None, 64)                2112      
                                                                 
 dense_145 (Dense)           (None, 32)                2080      
                                                                 
 dense_146 (Dense)           (None, 2)                 66        
                                                                 
Total params: 4,354
Trainable params: 4,354
Non-trainable params: 0
_________________________________________________________________


In [171]:
batch_size = 256
history = model.fit(
    x_train,
    y_train,
    epochs=25,# 300,
    batch_size=batch_size
)

Epoch 1/25
391/391 [==============================] - 1s 2ms/step - loss: 641.4860 - accuracy: 0.5425
Epoch 2/25
391/391 [==============================] - 1s 2ms/step - loss: 9684.6035 - accuracy: 0.5513
Epoch 3/25
391/391 [==============================] - 1s 2ms/step - loss: 39097.7188 - accuracy: 0.5513
Epoch 4/25
391/391 [==============================] - 1s 2ms/step - loss: 97615.7578 - accuracy: 0.5513
Epoch 5/25
391/391 [==============================] - 1s 2ms/step - loss: 193136.5156 - accuracy: 0.5513
Epoch 6/25
391/391 [==============================] - 1s 2ms/step - loss: 286089.1875 - accuracy: 0.5513
Epoch 7/25
391/391 [==============================] - 1s 2ms/step - loss: 446247.9062 - accuracy: 0.5513
Epoch 8/25
391/391 [==============================] - 1s 2ms/step - loss: 629875.3125 - accuracy: 0.5513
Epoch 9/25
391/391 [==============================] - 1s 2ms/step - loss: 815172.6250 - accuracy: 0.5513
Epoch 10/25
391/391 [==============================] - 1s 2ms/

In [69]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Accuracy: {:.1f}%'.format(acc*100))

gen = polar_generator(n_train+n_test,grid=(g1,g2),noise=0.02,flat=True)
for x in range(5):
  x,y = next(gen)

  score, acc = model.evaluate(x, y, batch_size=batch_size)
  print('Test score:', score)
  print('Accuracy: {:.1f}%'.format(acc*100))

79/79 [==============================] - 0s 3ms/step - loss: 4.6059 - accuracy: 0.0200
Test score: 4.605925559997559
Accuracy: 2.0%


ValueError: ignored